In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import time
import re
from progressbar import ProgressBar
from dateutil.parser import parse
import datetime

In [2]:
#Set headers
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}

# download url

In [5]:
import twint
import nest_asyncio
config = twint.Config()

d_begin = "2022-03-12"
d_end = "2022-04-02"
daily_datarange = pd.date_range(start=d_begin,
                                end=d_end ,
                                freq='b')

nest_asyncio.apply()
tweet_ids = []
tweet_urls  = []

for start_i in range(0, len(daily_datarange), 1):
    c = twint.Config()
    c.Username = "nytimes"   
    c.Pandas = True
    
    c.Since = str(daily_datarange[start_i].strftime("%Y-%m-%d"))
    c.Until = str((daily_datarange[start_i] + datetime.timedelta(days=2)).strftime("%Y-%m-%d"))
    c.Store_csv = True
    c.Custom_csv = ["id", "user_id", "username", "tweet"]
    c.Output = ("C:/Users/13a71/Dropbox/github/crawler/tweeter crawler/twint/nyt/{}.csv".format(c.Since))
    c.Filter_retweets = True
    # Run
    twint.run.Search(c)
    
    
    tweet_urls.extend([str(tweet.urls) for tweet in tweet_ids])


1503882500761956352 2022-03-16 08:55:03 +0900 <nytimes> Some players on the Mets and Yankees may be unable to play in New York when the 2022 MLB season begins next month because of a city vaccination mandate.  https://t.co/hLQHskYYKA
1503879982896005123 2022-03-16 08:45:03 +0900 <nytimes> A top Republican in the Ohio Senate race used offensive stereotypes about Asian people in 2013, citing a book that drew allegations of racism.  https://t.co/Hq6mkeMk0f
1503876218634051588 2022-03-16 08:30:05 +0900 <nytimes> At least nine House Democrats have said in the last five days that they tested positive for the coronavirus. More than half of those cases emerged after lawmakers attended a party retreat last week in Philadelphia.  https://t.co/NHm8UZe1uM
1503873723664941073 2022-03-16 08:20:11 +0900 <nytimes> In 1960, a child’s body was found in Arizona. Using DNA technology, investigators have finally identified her as Sharon Lee Gallegos, age 4.  It is the oldest cold case that the National Cen

# load url
- 지금은 url을 별도 폴더에 r에서 받아와서 불러오고 있다.
- 차후 수정

In [7]:
searchlist = pd.read_excel('~/documents/nyt_raw_tweet0402.xlsx').urls

In [8]:
#Open/reset progressbar and dataframes
pbar = ProgressBar()

#Create dataframe for storing article contents
content_df = pd.DataFrame(columns=['date', 'title','category', 'author', 'content', 'url'])

#Create dataframe for errorlist
error_df = pd.DataFrame(columns=['url'])

#Set headers
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}

In [9]:
import re
import requests
from datetime import datetime
from bs4 import BeautifulSoup, Tag
from nltk.tokenize import sent_tokenize
from time import gmtime, strftime

def now():
    """
    Returns
    -------
    Current time : str
        eg: 2018-11-22 13:35:23
    """
    return strftime("%Y-%m-%d-%W %H:%M:%S", gmtime())

def get_soup(url, headers=None):
    """
    Arguments
    ---------
    url : str
        Web page url
    headers : dict
        Headers for requests. If None, use Mozilla/5.0 as default user-agent

    Returns
    -------
    soup : bs4.BeautifulSoup
        Soup format web page
    """

    if headers is None:
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
    r = requests.get(url, headers=headers)
    html = r.text
    page = BeautifulSoup(html, 'lxml')
    return page

doublespace_pattern = re.compile('\s+')
lineseparator_pattern = re.compile('\n+')

def normalize_text(text):
    text = text.replace('\t', ' ')
    text = text.replace('\r', ' ')
    text = lineseparator_pattern.sub('\n', text)
    text = doublespace_pattern.sub(' ', text)
    text = ' '.join(sent_tokenize(text))
    return text.strip()


In [10]:
def parse_article(input_url):
    soup = get_soup(input_url)
        
    # get url in soup
    try:
        links_all= soup.find_all('link')
        links = [url['href'] for url in links_all if "nytimes" in url['href']]
        link = links[0]
    except:
        link = ''
            
    #Get Kicker (catergory)
    try:
        kicker = soup.find('span', class_='kicker-label').text
    except:
        try:
            kicker = link.split('/')[-2]
        except:
            print('No Kicker' + input_url)
            kicker = ""

    #Get the title
    try:
        title = soup.select('h1')[0].text
        #print(title)
    except:
        title =''

        #Try to get the subtitle[24:34]

        #Get date and strip
    try:
        date = soup.find('time')['datetime'][:10]
    except:
        try:
            raw_date = link[24:34]
            date = datetime.strptime(raw_date, "%Y/%m/%d")
        except:
            try:
                date = url[24:34]  
            except:
                print('No date'  + input_url)
                date = ""

    #Get author
    try:
        author = soup.select('p[itemprop^=author]')[0].text
    except:
        author =  ''

    #Get body text and strip advertisement
    try:
        content = ''
        body = content.join(p.text.replace(",", "") for p in soup.find_all('p')[2:-1])
    except:
        body = ""
        #print (body)
        
    json_object = {
        'date': date,
        'title': title,
        'subtitle': '',
        'content': body,
        'source': 'New York Times',
        'url': input_url,
        'category' : kicker
        }
    return json_object

In [11]:
import json 

def save(json_obj, directory):
    date = json_obj.get('date', '')
    title = json_obj.get('title', '')
    filepath = '{}/{}_{}.json'.format(directory, date, re.sub('[^a-zA-Z ]+',"", title[:50]))
    with open(filepath, 'w', encoding='utf-8') as fp:
        json.dump(json_obj, fp, indent=2, ensure_ascii=False)
        print('scraped {}'.format(json_obj['title'][:10]))
        
## CNN SCRAPER
directory = 'C:/Users/13a71/Documents/crawling output/nyt'
    
    
for url in pbar(searchlist):
    url = url[2:-2]
    try:
        article = parse_article(url)
        save(article, directory)
    except:
        print('this url is error')

scraped Fatal Texa
scraped 6 Injured 
scraped Russia Tar
scraped Russia Tar
scraped Kyiv’s Sub
scraped The Omicro
scraped A judge th
scraped What Happe


scraped What New Y
scraped How Long S
scraped Deshaun Wa
scraped On (Not) W
scraped Stocks not
scraped When Your 
scraped The Secret
scraped U.S. warns
scraped Mail Ballo


scraped How Long S
scraped What Happe
scraped What Happe
scraped Eugene N. 
scraped What Happe
scraped As diploma
scraped As Offices
scraped With Zero 
scraped What Happe


scraped Two Years 
scraped What Happe
scraped Asian Amer
scraped Who Should
scraped Where Stat
scraped A ‘Lost Ci
scraped Russia’s B
scraped What Happe
scraped Opponents 


scraped Nations Sh
scraped Hong Kong 
scraped What New Y
scraped Citizens o
scraped ‘American 
scraped Janet MacP
scraped China’s In
scraped School Vid
scraped John Clayt


...

scraped The Next B
scraped Imran Khan
scraped What Happe
scraped Russia’s W
this url is error
scraped House Pass
scraped Arizona Pa
scraped Will Endin
scraped Seven Deca


scraped Biden will
scraped Judge Toss
scraped Shaken at 
scraped Russia’s A
scraped Netrebko L
scraped The Grammy
scraped Fires, The
scraped Jerrod Car
scraped With Defen


scraped Harry Styl
scraped Ukraine Li
scraped Yale Museu
scraped What Happe
scraped 10 New Boo
scraped Cuomo Sues
scraped Cannabis f
scraped U.S. Will 
scraped South Caro


scraped How Low Wi
scraped This Man’s
scraped Sarah Pali
scraped Living Wit


scraped He Was Dem


In [12]:
content_df

,date,title,category,author,content,url


In [ ]:
content_df1 = content_df.loc[content_df["date"] >= "2020-10-12"].reset_index(drop=True)
content_df1 = content_df1.loc[content_df1["date"] <= "2021-11-13"].reset_index(drop=True)

In [ ]:
content_df

In [ ]:
content_df1

In [13]:
content_df.dropna(subset = ["date"], inplace=True)

In [14]:
content_df = content_df.drop_duplicates(subset='title')

In [15]:
content_df.to_excel('nytimes_20220402.xlsx')


## temp aggregate old

In [ ]:
nyt_old = nyt_old[['date', 'title', 'category', 'author', 'content', 'url']]

In [ ]:
content_df

In [ ]:
nyt = pd.concat([nyt, content_df])

In [ ]:
content_df = nyt

In [ ]:
content_df = nyt.drop_duplicates(subset='url')

In [ ]:
content_df['date'].unique()

In [ ]:
content_df['date'][0]

In [ ]:
[parse(str(raw_date)).strftime("%Y-%m-%d") for raw_date in content_df['date']]

In [ ]:

content_df.to_excel('nyt_new.xlsx')
error_df.to_excel('nyt_peace_errors.xlsx')nytimes1

In [ ]:
nyt = pd.read_excel('C:/Users/13a71/Dropbox/github/crawler/tweeter crawler/nytimes1.xlsx')

In [ ]:
nyt = pd.concat([nyt, content_df])

In [ ]:
from collections import Counter
from time import sleep, gmtime, strftime
content_df['date'] = content_df['date'].strftime("%Y-%m-%d")
Counter(content_df['date']).most_common()